# 3. Feature Engineering

In [1]:
import numpy as np

import pandas as pd
pd.options.display.max_columns = None

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

from IPython.display import Image

In [2]:
labels_df = pd.read_csv('../data/labels.csv')

In [3]:
master_df = pd.DataFrame()
master_df['BuildingID'] = labels_df['BuildingID']
master_df['Blighted'] = labels_df['Blighted']

## Features from Violation Dataset

In [4]:
violations = pd.read_csv('../data/violations.csv')
violations.head(2)

C:\Users\sumit\Anaconda3\envs\sdcnd\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (6,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,TicketID,TicketNumber,AgencyName,ViolName,ViolationStreetNumber,ViolationStreetName,MailingStreetNumber,MailingStreetName,MailingCity,MailingState,MailingZipCode,NonUsAddressCode,Country,TicketIssuedDT,TicketIssuedTime,HearingDT,CourtTime,ViolationCode,ViolDescription,Disposition,FineAmt,AdminFee,LateFee,StateFee,CleanUpCost,JudgmentAmt,PaymentStatus,Void,ViolationCategory,ViolationAddress,MailingAddress,unique_id,Coordinates,Latitude,Longitude,BuildingID
0,26288,05000001DAH,Department of Public Works,"Group, LLC, Grand Holding",2566,GRAND BLVD,743,"Beaubien, Ste. 201",Detroit,MI,48226,NaN,NaN,01/01/38440 12:00:00 AM,12:00:00,01/01/38474 12:00:00 AM,9:00AM,22-2-20,Burning solid waste in open fires,Responsible By Determination,$1500.00,$20.00,$150.00,$10.00,$0.00,$1680.00,PAID IN FULL,0.0,0,"2566 GRAND BLVD\r\nDetroit, MI\r\n(42.36318237...","743 Beaubien\r\nDetroit, MI 48226\r\n(42.33373...",7133,"(42.36318237000006, -83.09167672099994)",42.363182,-83.091677,dpsbvpy
1,19800,05000025DAH,Department of Public Works,"JACKSON, RAECHELLE",19014,ASHTON,20501,HEYDEN,DETROIT,MI,48219,NaN,NaN,01/01/38383 12:00:00 AM,10:15:00,01/01/38425 12:00:00 AM,1:30PM,22-2-22,Bulk solid waste deposited more than 24 hours ...,Not responsible By Determination,$100.00,$20.00,$10.00,$10.00,$0.00,$140.00,NO PAYMENT APPLIED,0.0,0,"19014 ASHTON\r\nDetroit, MI\r\n(42.42939076200...","20501 HEYDEN\r\nDETROIT, MI 48219\r\n(42.44217...",7134,"(42.429390762000025, -83.22039357799997)",42.429391,-83.220394,dpsc6k0


In [5]:
def remove_dollar_sign(s):
    return float(str(s).replace('$', ''))

violations['CleanJudgmentAmt'] = violations['JudgmentAmt'].map(remove_dollar_sign)
judgement_amt = violations[['BuildingID','CleanJudgmentAmt']].groupby('BuildingID', as_index=False).sum()
judgement_amt['CleanJudgmentAmt'].fillna(judgement_amt['CleanJudgmentAmt'].mean(), axis=0, inplace=True)

master_df = master_df.join(judgement_amt.set_index('BuildingID'), on='BuildingID')
master_df['CleanJudgmentAmt'].fillna(0, axis=0, inplace=True)

In [6]:
violations['ViolationCategory'].value_counts()

0    305774
1      2017
Name: ViolationCategory, dtype: int64

In [7]:
violations['PaymentStatus'].value_counts()

NO PAYMENT APPLIED      244290
PAID IN FULL             44319
NO PAYMENT ON RECORD     14565
PARTIAL PAYMENT MADE      4617
Name: PaymentStatus, dtype: int64

In [8]:
cat0 = violations[['BuildingID','ViolationCategory']][violations['ViolationCategory']==0].groupby('BuildingID', as_index=False).count()
cat0.columns = ['BuildingID','#ViolationCategory0']
cat1 = violations[['BuildingID','ViolationCategory']][violations['ViolationCategory']==1].groupby('BuildingID', as_index=False).count()
cat1.columns = ['BuildingID','#ViolationCategory1']

master_df = master_df.join(cat0.set_index('BuildingID'), on='BuildingID')
master_df = master_df.join(cat1.set_index('BuildingID'), on='BuildingID')
master_df['#ViolationCategory0'].fillna(0, axis=0, inplace=True)
master_df['#ViolationCategory1'].fillna(0, axis=0, inplace=True)

In [9]:
violation_payment_status_dict = {}

for val in violations['PaymentStatus'].unique():
    violation_payment_status_dict[val] = '#'+ val

for x in violation_payment_status_dict:
    cat = violations[['BuildingID','PaymentStatus']][violations['PaymentStatus']==x].groupby('BuildingID', as_index=False).count()
    cat.columns = ['BuildingID',violation_payment_status_dict[x]]
    master_df = master_df.join(cat.set_index('BuildingID'), on='BuildingID')
    master_df[violation_payment_status_dict[x]].fillna(0, axis=0, inplace=True)

In [10]:
count = violations[['BuildingID', 'unique_id']].groupby('BuildingID', as_index=False).count()
count.columns = ['BuildingID','#Violation']

master_df = master_df.join(count.set_index('BuildingID'), on='BuildingID')
master_df['#Violation'].fillna(0, axis=0, inplace=True)

In [11]:
master_df.head(2)

,BuildingID,Blighted,CleanJudgmentAmt,#ViolationCategory0,#ViolationCategory1,#NO PAYMENT APPLIED,#NO PAYMENT ON RECORD,#PAID IN FULL,#PARTIAL PAYMENT MADE,#Violation
0,dpt10mx,1,11405.0,30.0,0.0,24.0,0.0,5.0,1.0,30.0
1,dpscpjv,1,11105.0,31.0,0.0,31.0,0.0,0.0,0.0,31.0


## Features from 311 Calls

In [12]:
calls = pd.read_csv('../data/calls.csv')
calls.head(2)

,ticket_id,city,issue_type,ticket_status,issue_description,rating,ticket_closed_date_time,acknowledged_at,ticket_created_date_time,ticket_last_updated_date_time,address,lat,lng,location,image,unique_id,BuildingID
0,1516722.0,City of Detroit,Clogged Drain,Acknowledged,"Two drains one on each side of street, street ...",3,NaN,03/06/2015 10:03:38 PM,03/06/2015 09:57:52 PM,04/12/2015 01:01:10 AM,"13120-13130 Ilene Street Detroit, MI 48238, USA",42.383998,-83.16103856417159,"(42.3839977668, -83.1610385642)",NaN,434868.0,dpsc5eg
1,1525361.0,City of Detroit,Clogged Drain,Acknowledged,standing water on lumplin,2,NaN,03/11/2015 04:23:11 PM,03/11/2015 04:14:29 PM,04/07/2015 02:04:44 PM,"1485 E. Outer Drive Detroit, Michigan",42.440471,-83.080919,"(42.4404708, -83.080919)",NaN,434869.0,dpscmqn


In [13]:
calls = calls[calls['ticket_status'] != '06/22/2015 05:11:47 PM']

In [14]:
calls['ticket_status'].value_counts()

Archived        9592
Closed          7007
Acknowledged    3010
Open              59
Name: ticket_status, dtype: int64

In [15]:
calls['issue_type'].value_counts()

Illegal Dumping / Illegal Dump Sites                                                         3581
Tree Issue                                                                                   3545
Running Water in a Home or Building                                                          2653
Clogged Drain                                                                                2490
Potholes                                                                                     2398
Traffic Sign Issue                                                                           1030
Water Main Break                                                                              776
Fire Hydrant Issue                                                                            677
Abandoned Vehicle                                                                             638
Manhole Cover Issue                                                                           546
Traffic Signal Issue

In [16]:
calls['rating'].value_counts()

2     9182
3     7852
4     2059
5      417
6       79
1       54
7       20
8        3
19       1
Name: rating, dtype: int64

In [17]:
calls = calls[calls['rating'] != 19]

In [18]:
call_ticket_status_dict = {}

for val in calls['ticket_status'].unique():
    call_ticket_status_dict[val] = '#'+ val

for x in call_ticket_status_dict:
    cat = calls[['BuildingID','ticket_status']][calls['ticket_status']==x].groupby('BuildingID', as_index=False).count()
    cat.columns = ['BuildingID',call_ticket_status_dict[x]]
    master_df = master_df.join(cat.set_index('BuildingID'), on='BuildingID')
    master_df[call_ticket_status_dict[x]].fillna(0, axis=0, inplace=True)

In [19]:
call_issue_type_dict = {}

for val in calls['issue_type'].unique():
    call_issue_type_dict[val] = '#'+ val

for x in call_issue_type_dict:
    cat = calls[['BuildingID','issue_type']][calls['issue_type']==x].groupby('BuildingID', as_index=False).count()
    cat.columns = ['BuildingID',call_issue_type_dict[x]]
    master_df = master_df.join(cat.set_index('BuildingID'), on='BuildingID')
    master_df[call_issue_type_dict[x]].fillna(0, axis=0, inplace=True)

In [20]:
calls['rating'] = calls['rating'].astype(float)
rating = calls[['BuildingID', 'rating']].groupby('BuildingID', as_index=False).agg('mean')
count.columns = ['BuildingID','MeanCallRating']

master_df = master_df.join(count.set_index('BuildingID'), on='BuildingID')
master_df['MeanCallRating'].fillna(0, axis=0, inplace=True)

In [21]:
count = calls[['BuildingID', 'unique_id']].groupby('BuildingID', as_index=False).count()
count.columns = ['BuildingID','#Calls']

master_df = master_df.join(count.set_index('BuildingID'), on='BuildingID')
master_df['#Calls'].fillna(0, axis=0, inplace=True)

In [22]:
master_df.head(10)

,BuildingID,Blighted,CleanJudgmentAmt,#ViolationCategory0,#ViolationCategory1,#NO PAYMENT APPLIED,#NO PAYMENT ON RECORD,#PAID IN FULL,#PARTIAL PAYMENT MADE,#Violation,#Acknowledged,#Open,#Archived,#Closed,#Traffic Signal Issue,#Street Light Pole Down,#Running Water in a Home or Building,#Abandoned Vehicle,#Trash Issue - Bulk waste deposited more than 24 hours before designated time,#Illegal Dumping / Illegal Dump Sites,"#Customer Service (internal use only, private issue)",#Curbside Solid Waste Issue,#Potholes,#DPW - Debris Removal,#Clogged Drain,#Water Main Break,#Graffiti,"#Test (internal use only, public issue)",#Tree Issue,"#Graffiti Abatement (internal use only, public issue)",#Trash Issue - Improper placement of refuse container between collections/left at curbside,#Fire Hydrant Issue,#DPW - Other environmental,#Detroit Land Bank Referral,#Manhole Cover Issue,#Residential Snow Removal Issue,#Traffic Sign Issue,MeanCallRating,#Calls
0,dpt10mx,1,11405.0,30.0,0.0,24.0,0.0,5.0,1.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0
1,dpscpjv,1,11105.0,31.0,0.0,31.0,0.0,0.0,0.0,31.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,2.0
2,dpt10x2,1,13065.0,23.0,0.0,23.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,0.0
3,dpt12rp,1,9975.0,19.0,0.0,15.0,0.0,4.0,0.0,19.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,1.0
4,dpsbebm,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,dpt10qz,1,6820.0,22.0,0.0,17.0,1.0,4.0,0.0,22.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,1.0
6,dpt10rn,1,21062.5,32.0,0.0,31.0,0.0,1.0,0.0,32.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,4.0
7,dpt10b8,1,21232.5,56.0,0.0,44.0,2.0,7.0,3.0,56.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56.0,1.0
8,dpt10qb,1,7362.5,30.0,0.0,27.0,0.0,3.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0
9,dpsb7v8,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Features from Crimes' Report

In [23]:
crimes = pd.read_csv('../data/crimes.csv')
crimes.head(2)

C:\Users\sumit\Anaconda3\envs\sdcnd\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ROWNUM,CASEID,INCINO,CATEGORY,OFFENSEDESCRIPTION,STATEOFFENSEFILECLASS,INCIDENTDATE,HOUR,SCA,PRECINCT,COUNCIL,NEIGHBORHOOD,CENSUSTRACT,ADDRESS,LON,LAT,LOCATION,unique_id,Coordinates,Latitude,Longitude,BuildingID
0,53256,1953933,1506030028.1,ASSAULT,ASSAULT AND BATTERY/SIMPLE ASSAULT,13001.0,06/03/2015 12:00:00 AM,2,1007.0,10.0,City Council District 5,PETOSKEY-OTSEGO,5334.0,09100 PETOSKEY,-83.1221,42.3678,"PETOSKEY\r\n09100\r\n(42.3676, -83.1219)",314937,"(42.3676, -83.1219)",42.3676,-83.1219,dpsch8b
1,17631,1917717,1503010158.1,LARCENY,LARCENY - PARTS AND ACCESSORIES FROM VEHICLE,23006.0,03/01/2015 12:00:00 AM,9,608.0,6.0,City Council District 7,GRANDALE,5452.0,00 PLYMOUTH AND MANSFIELD,-83.2035,42.3724,"00 PLYMOUTH AND MANSFIELD\r\n(42.3725, -83.2033)",314938,"(42.3725, -83.2033)",42.3725,-83.2033,dpsc49t


In [24]:
crimes['CATEGORY'].value_counts()

TRAFFIC VIOLATIONS-MOTORCYCLE VIOLATIONS       29257
ASSAULT                                        16410
LARCENY                                        14106
DAMAGE TO PROPERTY                              9394
AGGRAVATED ASSAULT                              8256
BURGLARY                                        7746
STOLEN VEHICLE                                  6940
TRAFFIC VIOLATIONS-DRIVING ON SUSPENDED         5739
FRAUD                                           4891
ROBBERY                                         3179
DANGEROUS DRUGS                                 3101
OBSTRUCTING JUDICIARY                           1397
WEAPONS OFFENSES                                1254
ESCAPE                                           906
SOLICITATION                                     846
BRIBERY                                          832
ARSON                                            785
OUIL DISPOSE OF VEHICLE TO AVOID FORFEITURE      748
RUNAWAY                                       

In [25]:
crime_category_types = {}

for val in crimes['CATEGORY'].unique():
    crime_category_types[val] = '#'+ val

for x in crime_category_types:
    cat = crimes[['BuildingID','CATEGORY']][crimes['CATEGORY']==x].groupby('BuildingID', as_index=False).count()
    cat.columns = ['BuildingID',crime_category_types[x]]
    master_df = master_df.join(cat.set_index('BuildingID'), on='BuildingID')
    master_df[crime_category_types[x]].fillna(0, axis=0, inplace=True)

In [26]:
count = crimes[['BuildingID', 'unique_id']].groupby('BuildingID', as_index=False).count()
count.columns = ['BuildingID','#Crimes']

master_df = master_df.join(count.set_index('BuildingID'), on='BuildingID')
master_df['#Crimes'].fillna(0, axis=0, inplace=True)

In [27]:
master_df.head(1)

,BuildingID,Blighted,CleanJudgmentAmt,#ViolationCategory0,#ViolationCategory1,#NO PAYMENT APPLIED,#NO PAYMENT ON RECORD,#PAID IN FULL,#PARTIAL PAYMENT MADE,#Violation,#Acknowledged,#Open,#Archived,#Closed,#Traffic Signal Issue,#Street Light Pole Down,#Running Water in a Home or Building,#Abandoned Vehicle,#Trash Issue - Bulk waste deposited more than 24 hours before designated time,#Illegal Dumping / Illegal Dump Sites,"#Customer Service (internal use only, private issue)",#Curbside Solid Waste Issue,#Potholes,#DPW - Debris Removal,#Clogged Drain,#Water Main Break,#Graffiti,"#Test (internal use only, public issue)",#Tree Issue,"#Graffiti Abatement (internal use only, public issue)",#Trash Issue - Improper placement of refuse container between collections/left at curbside,#Fire Hydrant Issue,#DPW - Other environmental,#Detroit Land Bank Referral,#Manhole Cover Issue,#Residential Snow Removal Issue,#Traffic Sign Issue,MeanCallRating,#Calls,#EMBEZZLEMENT,#CONSPIRACY BY COMPUTER,#OUIL DISPOSE OF VEHICLE TO AVOID FORFEITURE,#GAMBLING,#FELONY DEATH FROM FLEEING VEHICLE,#DRUNKENNESS,#OBSTRUCTING JUDICIARY,#IMMIGRATION,#OUIL,#JUSTIFIABLE HOMICIDE,#FAMILY OFFENSE,#HOMICIDE,#NEGLIGENT HOMICIDE,#OBSTRUCTING THE POLICE,#ELECTION LAWS,#PUBLIC PEACE,#ASSAULT,#SOLICITATION,#VAGRANCY (OTHER),#REVOKED,#KIDNAPING,#AGGRAVATED ASSAULT,#EXTORTION,#TRAFFIC VIOLATIONS-MOTORCYCLE VIOLATIONS,#OBSCENITY,#TRAFFIC VIOLATIONS-DRIVING ON SUSPENDED,#WEAPONS OFFENSES,#ANTITRUST,#DANGEROUS DRUGS,#LARCENY,#ROBBERY,#RUNAWAY,#DAMAGE TO PROPERTY,#BRIBERY,#LIQUOR,#STOLEN PROPERTY,#ESCAPE,#ENVIRONMENT,#FORGERY,#HEALTH-SAFETY,#TAX REVENUE,#BURGLARY,#MISCELLANEOUS ARREST,#CONGRESS,#OTHER BURGLARY,#MILITARY,#ARSON,#SOVEREIGNTY,#STOLEN VEHICLE,#FRAUD,#Crimes
0,dpt10mx,1,11405.0,30.0,0.0,24.0,0.0,5.0,1.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Saving Dataset

In [28]:
master_df.to_csv('../data/master_dataset.csv', index=False, encoding='utf-8')